In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Model Implementation
class MF(nn.Module):
    def __init__(self, num_factors, num_users, num_items):
        super(MF, self).__init__()
        self.P = nn.Embedding(num_embeddings=num_users, embedding_dim=num_factors)
        self.Q = nn.Embedding(num_embeddings=num_items, embedding_dim=num_factors)
        self.user_bias = nn.Embedding(num_embeddings=num_users, embedding_dim=1)
        self.item_bias = nn.Embedding(num_embeddings=num_items, embedding_dim=1)

    def forward(self, user_id, item_id):
        P_u = self.P(user_id)
        Q_i = self.Q(item_id)
        b_u = self.user_bias(user_id).squeeze()
        b_i = self.item_bias(item_id).squeeze()
        outputs = (P_u * Q_i).sum(1) + b_u + b_i
        return outputs

# RMSE Evaluation
def evaluator(net, test_iter, device):
    criterion = nn.MSELoss()
    net.eval()  # Set the model to evaluation mode
    rmse = 0
    count = 0
    with torch.no_grad():
        for users, items, ratings in test_iter:
            users, items, ratings = users.to(device), items.to(device), ratings.to(device)
            preds = net(users, items)
            loss = torch.sqrt(criterion(preds, ratings.float()))
            rmse += loss.item() * len(ratings)
            count += len(ratings)
    return rmse / count

# Training and Evaluating the Model
def train_recsys_rating(net, train_iter, test_iter, loss_fn, optimizer, num_epochs, device):
    net = net.to(device)
    for epoch in range(num_epochs):
        net.train()  # Set the model to training mode
        train_loss = 0
        for users, items, ratings in train_iter:
            users, items, ratings = users.to(device), items.to(device), ratings.to(device)

            optimizer.zero_grad()
            preds = net(users, items)
            loss = loss_fn(preds, ratings.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        test_rmse = evaluator(net, test_iter, device)
        print(f'Epoch {epoch + 1}, Train Loss: {train_loss / len(train_iter):.4f}, Test RMSE: {test_rmse:.4f}')

# Example usage
num_users, num_items, num_factors = 1000, 1000, 10
net = MF(num_factors=num_factors, num_users=num_users, num_items=num_items)

# Example data (replace with actual data)
users = torch.randint(0, num_users, (1000,))
items = torch.randint(0, num_items, (1000,))
ratings = torch.randint(1, 6, (1000,))

train_dataset = TensorDataset(users[:800], items[:800], ratings[:800])
test_dataset = TensorDataset(users[800:], items[800:], ratings[800:])

train_iter = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_iter = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_fn = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

train_recsys_rating(net, train_iter, test_iter, loss_fn, optimizer, num_epochs=5, device=device)

# Predict the rating that user ID 20 might give to item ID 30
user_id = torch.tensor([20]).to(device)
item_id = torch.tensor([30]).to(device)
net.eval()  # Set the model to evaluation mode
with torch.no_grad():
    score = net(user_id, item_id)
print(f'Predicted rating for user 20 and item 30: {score.item()}')


Epoch 1, Train Loss: 23.9337, Test RMSE: 4.9090
Epoch 2, Train Loss: 19.2723, Test RMSE: 4.8857
Epoch 3, Train Loss: 16.0437, Test RMSE: 4.8655
Epoch 4, Train Loss: 13.1306, Test RMSE: 4.8488
Epoch 5, Train Loss: 10.5954, Test RMSE: 4.8425
Predicted rating for user 20 and item 30: -2.2230846881866455
